In [1]:
import open3d as o3d
import numpy as np 
from matplotlib import pyplot as plt

VOXEL_SIZE = 20

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load point clouds
pcd_raw = [o3d.io.read_point_cloud('./view_1_rgb_pcd_light_obj_dectection_colour.ply'), 
           o3d.io.read_point_cloud('./view_2_rgb_pcd_light_obj_dectection_colour.ply'),
           o3d.io.read_point_cloud('./view_3_rgb_pcd_light_obj_dectection_colour.ply'),
           o3d.io.read_point_cloud('./view_4_rgb_pcd_light_obj_dectection_colour.ply'),
           o3d.io.read_point_cloud('./view_5_rgb_pcd_light_obj_dectection_colour.ply'),
           o3d.io.read_point_cloud('./view_6_rgb_pcd_light_obj_dectection_colour.ply')]

# Downsample point clouds
pcd_ds = []
for pcd in pcd_raw:
    pcd_ds.append(pcd.voxel_down_sample(voxel_size=VOXEL_SIZE))

In [3]:
# Reference https://github.com/isl-org/Open3D/issues/5501
def get_clusters(pcd, apply_label_colors = False, compute_aabbs = True, eps=500, min_points=10):
        pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20,std_ratio=2.0)
        labels = np.array(
            pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))

        if labels.size == 0:
            return None, None

        max_label = labels.max()
        
        if apply_label_colors:
            colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
            colors[labels < 0] = 0
            pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
        
        clusters = [pcd.select_by_index(list(np.where(labels == i)[0])) for i in range(max_label + 1)]
        aabbs    = [cluster.get_axis_aligned_bounding_box() for cluster in clusters] if compute_aabbs else []

        for aabb in aabbs:
            aabb.color = [0, 0, 0]

        return clusters, aabbs

In [5]:
pcd_crp = []
for k, pcd in enumerate(pcd_ds):
    clusters, aabbs = get_clusters(pcd)
    pcd_crp.append(pcd_raw[k].crop(aabbs[0]))
o3d.visualization.draw_geometries(pcd_crp)

In [ ]:
#o3d.io.write_point_cloud('./view_6_rgb_pcd_light_obj_dectection_colour_cropped.ply', pcd_crp[5])

True